In [3]:
import json
import numpy as np 
threshold = 0.6
names = []

path = '/home/banyh2000/odfn/pics/injection/output_functional_8.json'
with open(path) as f:
    data = json.load(f)
    data = np.array(data)
length = data.shape[0]
print(np.sum(data>threshold)/length)


[0.3504512  0.48509438 0.36470252 1.         0.         0.51344352
 0.61762681 0.5028552  0.29466433 0.06725873 0.42217739 0.
 0.4300052  0.98972877 1.         1.         0.5677052  0.50521544
 0.96992207 0.64110576 0.61183914 0.         0.49972833 0.
 1.         0.2861607  0.71184804 1.         0.58793416 0.
 0.         0.         0.65305414 0.01669635 0.96257889 0.55361552
 1.         0.74929229 0.59203386 0.4121774  0.06421575 0.44933859
 0.42397526 0.43155767 1.         0.         0.41495434 0.72751093
 0.84251261 1.         1.         0.36737054 0.66981039 0.97893739
 0.57094593 1.         0.48807637 1.         0.79283718 0.97603979
 0.06259982 0.09914771 0.         0.50966846 0.         0.43700455
 0.72193654 0.58157994 0.92222516 0.0544924  0.8160162  0.46337633
 0.         0.         1.         0.97240909 0.69302141 0.95138407
 0.8695701  0.55759672 0.35957897 0.         1.         0.37219554
 0.18925291 0.2688395  0.42188182 0.24959503 0.99138292 0.46410745
 0.53172604 1.     